# Burrows Wheeler Transform
### Sergio Peignier (sergio.peignier@insa-lyon.fr)

In this practical work, we will program and study the Burrows Wheeler transform

### Lexicographic order

In this work, we compare two strings iteratively, by comparing their characters by their so-called lexicographic order, (i.e., their alphabetic order A<B<C<....<Z).
Notice that here we consider only the alphabet characters, and an "end of string" character, denoted \\$ s.t. \\$ < A.
Therefore, a string $S$ that is a prefix of a string $T$ is considered smaller: $S<T$.

### Circular Shift

Let $R(T,k)=T'$ be a function that takes a string $T$ and a integer $k$ as inputs and returns a string $T'$ that is a circular shift of string $T$, such that: 

$T'[1,|T|-k] = T[k+1,|T|]$ and $T'[|T|-k + 1,|T|] = T[1,k]$

__example__:

for T = "dans l'herbe noire Les Kobolds vont$"

R(T,3) = "s l'herbe noire Les Kobolds vont$dan"

R(T,9) = "rbe noire Les Kobolds vont$dans l'he"

+ Program function $R(T,k)$ in python

### Burrows Wheeler Transform (BWT)
+ To compute the Burrows Wheeler Transform (BWT) we must:
    + Assume that the string contains a special character "End of string" that only occurs at the end of the end of the string and nowhere else (here: "\$")
    + Apply the function $R(T,k), \quad \forall k\in [1,...,|T|]$ and store the outputs (i.e., compute and store all the circular shifts)
    + Sort the strings into lexical order
    + Extract the last letter of each string 
    
__Example__:

Original String: $S=$ ACATACAGATG\$

Sorted circular shifts:

\$ACATACAGATG

ACAGATG\$ACAT

ACATACAGATG\$

AGATG\$ACATAC

ATACAGATG\$AC

ATG\$ACATACAG

CAGATG\$ACATA

CATACAGATG\$A

G\$ACATACAGAT

GATG\$ACATACA

TACAGATG\$ACA

TG\$ACATACAGA

Burrow Wheeler Transform: 

$BWT(S) = $ GT\$CCGAATAAA

__Questions :__

+ Program the BWT in python
+ Apply the BWT to encode the virus genome downloaded during the previous lecture

In [1]:
test="hello"
print(test[2:])

llo


In [2]:
def circular_shift(text,k):
    """
    Make a circular shift
    
    Args:
        text (str): string to be shifted

    Returns:
        str: shifted string
    """
    shifted_text =""
    #dans le cas où k>taille du texte
    shift=k%len(text)
    
    shifted_text=text[:len(text)-shift]
    i=0
    while shift>i:
        shifted_text=text[len(text)-1-i]+shifted_text
        i+=1
        
    return shifted_text

def BWT(text,verbose=False):
    """
    Compute the BWT
    
    Args:
        text (str): string to be shifted
        verbose (bool): print sorted circular shifts if true
    
    Return:
        str: BWT
    """
    bwt = ""
    new_text=text+"$"
    list_shifts=[]
    for k in range(len(new_text)):
        shift=circular_shift(new_text,k)
        list_shifts.append(shift)
    list_shifts.sort()
    for element in list_shifts:
        bwt+=element[-1]
        
    return(bwt)


In [3]:
text = "dans l'herbe noire les kobolds vont. le vent profond pleure, on veut croire."

In [4]:
shifted_text = circular_shift(text,10)
if shifted_text == "ut croire.dans l'herbe noire les kobolds vont. le vent profond pleure, on ve":
    print("well done!")
else:
    print("wrong answer")

well done!


In [5]:
burrough = BWT(text)
print(burrough)
if burrough == "tss.ee,dtens.letedro n$lrblrrvhllvo'oo  o  poo aeokrnrb fv  eiuipcendunnee   ":
    print("well done!")
else:
    print('wrong answer')

tss.ee,dtens.letedro n$lrblrrvhllvo'oo  o  poo aeokrnrb fv  eiuipcendunnee   
well done!


In [6]:
BWT("ACATACAGATG",verbose=True)

'GT$CCGAATAAA'


### Data Compression 
The BWT is a algorithm that aims at preparing data to be compressed.
In practice it rearranges a string into pieces of repeated characters, that can then be compressed using for example the __run-length encoding__.
This technique aims at storing a sequence of repeated characters as a single value and the number of repeats.

__Example__

_original string_ : AAAAAAATTTTTGGGGTGTTTTTT 

_compressed string_ : A7T5G4TGT6

__Questions :__

+ Write a simple function that performs the Run length encoding
+ Estimate the complexity of your function
+ Compress the genome before and after applying the BWT

In [7]:
def run_length_encoding(S):
    """
    Encode sequence using the Run Length method
    
    Args:
        text (str): string to be shifted
    
    Return:
        str: run length
    """
    encoded_S= ""
    i=0
    number=1
    while i<len(S):
        encoded_S+=S[i]
        i+=1
        while i<len(S) and S[i-1]==S[i]:
            number+=1
            i+=1
        if number>1:
            encoded_S+=str(number)
        number=1
    return encoded_S

In [8]:
if run_length_encoding("AAAATTCCCGCGG") == 'A4T2C3GCG2':
    print("well done!")
else:
    print("wrong answer")
print(run_length_encoding("AAAATTCCCGCGG"))

well done!
A4T2C3GCG2



### BWT and suffix tables: Understanding the BWT
Let $S$ be a string, and let $S'$ be the transformed version of $S$ after undergoing a given number of circular shift operations, and let us consider that the end of string character is at location $i$. 

__Questions :__
+ What does the substring $S[1:i]$ represent for the substring $S[i+1:|S|]$, and particularly for the last character $S[|S|]$?
+ Describe the BWT in terms of suffixes
+ Assuming that there are some regularities in the string (some elements tend to be followed by some particular elements with higher frequency), explain why the BWT tends to bring together the same characters.
+ Conceive an algorithm that takes a string and its suffix table as an input, and returns its BWT.
+ Compute the complexity of this method


In [9]:
def suffix_list(T):
    """
    Compute the suffix list
    
    Args:
        T (str): string
    
    Return:
        list of strings: suffix list
    """
    suffix_list = [T[i:] for i in range(len(T))]
    sorted(suffix_list,reverse=True)
    return suffix_list

def suffix_table(T):
    """
    Compute the suffix table
    
    Args:
        T (str): string
    
    Return:
        list of tuples (suffix,location): suffix table
    """
    suffix_list = [T[i:] for i in range(len(T))]
    suffix_table = sorted((e,i) for i,e in enumerate(suffix_list))
    return suffix_table

def BWT_suffix_table(T,end_of_string="$"):
    """
    Compute the BWT from the suffix table
    
    Args:
        T (str): string
        end_of_string (char): end of string character to append
    
    Return:
        bwt (str): BWT
    """
    T += end_of_string
    ST = suffix_table(T)
    bwt = ""
    for s,i in ST:
        bwt += T[i-1]
    return(bwt)

In [10]:
T = "ACATACAGATG"
print(suffix_list(T))
print(suffix_table(T))

['ACATACAGATG', 'CATACAGATG', 'ATACAGATG', 'TACAGATG', 'ACAGATG', 'CAGATG', 'AGATG', 'GATG', 'ATG', 'TG', 'G']
[('ACAGATG', 4), ('ACATACAGATG', 0), ('AGATG', 6), ('ATACAGATG', 2), ('ATG', 8), ('CAGATG', 5), ('CATACAGATG', 1), ('G', 10), ('GATG', 7), ('TACAGATG', 3), ('TG', 9)]


In [11]:
T = "ACATACAGATG"
if BWT(T) == BWT_suffix_table(T):
    print('well done!')
else:
    print("wrong answer")
    
print(BWT_suffix_table(T))

well done!
GT$CCGAATAAA


### Naive Inverse Burrows Wheeler Transform
The BWT transformation is reversible, and the inverse operation does note require the storage of any additional data: only the end of string character is needed.
The inverse BWT procedure is described hereafter:

+ Let $S$ be the original string and let $BWT(S) = S_{bwt}$ (we only know $S_{bwt}$)
+ Given only $S_{bwt}$, we can easily reconstruct the first column of the circular shifts table: indeed,  $S_{bwt}$ contains all the characters of $S$, and we know that the first column also contains all the characters of $S$ sorted. Then, simply sort the characters of $S_{bwt}$ to get the first column. Let $S_1$ denote this first column.
+ Given the circular shift operation, we know that $\forall i$ if $S[i] = S_{bwt}[i]$, then $S[i+1] = S_{1}[i]$

+ Now, $\{\forall i \quad (S_{bwt}[i], S_{1}[i])\}$ denote the set of all pairs of successive characters in $S$.
+ Again, we know that the first and second columns of the table contain the pairs are successive characters sorted, then we simply need to sort these pairs to get the first and second column of the table
+ Similarly, we simply need to paste the characters in $S_{bwt}$ to those of the first two columns to get the set of all triplets of successive characters in $S$, and sort them to get the first three columns and so on, until reconstructing the entire table...
+ Once the table is reconstructed, the row containing the "end of string" character in the last position is the original string.

__Example :__

$S_{bwt}$ = GT$CCGAATAAA

Sort to get $S_1$: $S_1$ = $AAAAACGGTT

Paste $S_{bwt}$ to get the pairs: $\{\forall i \quad (S_{bwt}[i], S_{1}[i])\}$ = \{G\\$, TA, \\$A, CA, CA, GA, AC, AC, TG, AG, AT, AT\}

Sort the pairs to get the (first and) second column: \\$A, AC, AC, AG, AT, AT, CA, CA, G\\$, GA, TA, TG 

$S_{2}$ = ACCGTTAA\\$AAG

Paste $S_{bwt}$ to get the triplets: $\{\forall i \quad (S_{bwt}[i], S_{1}[i], S_{2}[i])\}$ = \{G\\$A, TAC, \\$A, CAG, CAT, GAT, ACA, ACA, TG\\$, AGA, ATA, ATG\}

...

__Questions :__

+ Program the inverse Burrow Wheeler Transform 
+ Compute the complexity of this algorithm

In [12]:
def print_table(table):
    for row in table:
        print("".join(row))

def inverse_BWT(bwt, verbose=False, last_character="$"):
    """
    Inverse the BWT
    
    Args:
        bwt (str): bwt of a string T
        verbose (bool): if True print the process of the algorithm
        last_character (char): which is the end of string character?
    
    Return:
        T (str): BWT^{-1} of bwt
    """
    T = ""
    grid=np.zeros((len(bwt),len(bwt)))
    for k in range(len(bwt)):
        grid[k][len(bwt)-1]=bwt[k]
        
    first_column=list(bwt)
    first_column.sort()
    
    for k in range(len(bwt)):
        grid[k][0]=first_column[k]
        
    pairs=[]
    for i in range(len(first_column)-1):
        for j in range(len(bwt)):
            if first_column[i]==bwt[j]:
                pairs.append((bwt[j],first_column[i+1]))
                grid[]
    
    return T

SyntaxError: invalid syntax (284785486.py, line 33)

In [13]:
if inverse_BWT(BWT(text)) == text:
    print("well done!")
else:
    print("wrong answer")

NameError: name 'inverse_BWT' is not defined

### Inverse Burrows Wheeler Transform
There is a more efficient way to perform the Inverse Burrows Wheeler Transform, without needing to reconstruct the entire table (pasting and sorting)

#### First Last (FL) property
+ Consider the following example. Index the letters of the Last column by the order of appearance of each letter (if you have seen k letters A then index the new letter A as A$_{k+1}$)
+ Search the correspondence of indexes in the First column, to do so, you can use the entire table 
+ What do you notice? Why does this property appears?

__Example:__

\\$ACATACAGATG

ACAGATG\$ACAT

ACATACAGATG\$

AGATG\$ACATAC

ATACAGATG\$AC

ATG\$ACATACAG

CAGATG\$ACATA

CATACAGATG\$A

G\$ACATACAGAT

GATG\$ACATACA

TACAGATG\$ACA

TG\$ACATACAGA

### Efficient Inverse Burrows Wheeler Transform

Let $S_{bwt}$ the BWT of a string $S$, and let $A$ be the alphabet of $S$.

Let $S*T = (S[1], S[2], \dots, S[|S|], T[1],  \dots, T[|T|])$ be the concatenation of strings $S$ and $T$ 

__initialization__

+ For each character $X \in A$ count its number of occurrences $\# X$ in $S_{bwt}$ 
+ Index the occurrences of $S_{bwt}$ by their order of appearance, i.e., the first occurrence of $X$ is indexed 1, the second is indexed 2, and so on (e.g., for the sequence (A,A,T,A,C,C) the indexes are (1,2,1,3,1,2)). 
Let $K$ represent the list of indexes, such that $K[i]$ is the index of $S_{bwt}[i]$
+ By definition, the first character of the First column is \\$, its left context is stored in the first character of the Last column ($S_{bwt}[0]$), therefore this is the penultimate character in $S$ (just before \\$). 

+ Let $X\leftarrow S_{bwt}[i]$ denote the current character
+ And let $k \leftarrow 1$ be its index (it is the first occurrence of $X$ in $S_{bwt}$)
+ $S \leftarrow (\$)$

__Repeat While__ $X \neq \$$

The main idea is to find the location of $X$ in the First column of the table (not computed explicitly), in order to find its left context in the Last column, i.e., $S_{bwt}$. Notice that the characters in the first column are: 1) sorted into a lexicographic order 2) the instances of the same letter share the same appearance index than those in $S_{bwt}$. 

+ $S \leftarrow (X)*S$
+ Let $j$ be the location of $X$ in the First column. $j\leftarrow k+\sum_{Y} \# Y $ for all character $Y \in A$ such that $Y<X$
+ The character at $S_{bwt}[j]$ has $X$ as right context, and thus it is its the left context.
+ $X \leftarrow S_{bwt}[j]$, $k \leftarrow K[j]$ 

__Questions:__

+ Implement this algorithm in python
+ What is the complexity of this algorithm?


In [62]:
def localisation_count(bwt):
    """ renvoie un tableau T tel que T[i] renvoie combien de fois bwt[i] est apparu dans bwt[:i]
    """
    localisation=[0 for i in range(len(bwt))]
    count=[0 for i in range(5)]
    for i in range(len(bwt)):
        if bwt[i]=='$':
            count[0]+=1
            localisation[i]=count[0]
            
        elif bwt[i]=='A':
            count[1]+=1
            localisation[i]=count[1]
            
        elif bwt[i]=='C':
            count[2]+=1
            localisation[i]=count[2]
            
        elif bwt[i]=='G':
            count[3]+=1
            localisation[i]=count[3]
            
        elif bwt[i]=='T':
            count[4]+=1
            localisation[i]=count[4]
            
    return localisation, count

def left_corresponding_letter(letter, alphabet, bwt, count, localisation, bwt_index):
        occurence=localisation[bwt_index]
        #find the position i in the first column
        position=sum(count[:alphabet[letter]])+occurence
        #find the letter on position i in the last column
        letter=bwt[position-1]
        return letter, position-1



def efficient_inverse_BWT(bwt,end_of_string="$"):
    """
    Inverse the BWT
    
    Args:
        bwt (str): bwt of a string T
        last_character (char): which is the end of string character?
    
    Return:
        T (str): BWT^{-1} of bwt
    """  
    T = str(end_of_string)

    
    

            
    localisation, count= localisation_count(bwt)       
    alphabet={'$':0, 'A':1,'C':2,'G':3, 'T':4}
    test=[0 for i in range(5)]
    letter=bwt[0]
    occurence=localisation[0]
    
    while letter!='$':
        T=letter+T
        test[alphabet[letter]]+=1
        
        print("letter is " +str(alphabet[letter])+ "th place in the alphabet")
        print(occurence)
        print("count table")
        print(count)
        
        #find the position i in the first column
        position=sum(count[:alphabet[letter]])+occurence
        
        print("position of letter in first column")
        print(position)
        
        #find the letter on position i in the last column
        letter=bwt[position-1]
        occurence=localisation[position-1]
        print("new letter is " +str(letter))
        
        
    return(T)

In [28]:
test=[1,2,3,4]
sum(test[:1])

1

In [32]:
if efficient_inverse_BWT(BWT(text)) == text:
    print("well done!")
else:
    print("wrong answer")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]


KeyError: 't'

In [60]:
if efficient_inverse_BWT("GT$CCGAATAAA") == "ACATACAGATG$":
    print("well done!")
else:
    print("wrong answer")
    print(efficient_inverse_BWT("GT$CCGAATAAA"))

letter is 3th place in the alphabet
1
count table
[1, 5, 2, 2, 2]
position of letter in first column
9
new letter is T
letter is 4th place in the alphabet
2
count table
[1, 5, 2, 2, 2]
position of letter in first column
12
new letter is A
letter is 1th place in the alphabet
5
count table
[1, 5, 2, 2, 2]
position of letter in first column
6
new letter is G
letter is 3th place in the alphabet
2
count table
[1, 5, 2, 2, 2]
position of letter in first column
10
new letter is A
letter is 1th place in the alphabet
3
count table
[1, 5, 2, 2, 2]
position of letter in first column
4
new letter is C
letter is 2th place in the alphabet
1
count table
[1, 5, 2, 2, 2]
position of letter in first column
7
new letter is A
letter is 1th place in the alphabet
1
count table
[1, 5, 2, 2, 2]
position of letter in first column
2
new letter is T
letter is 4th place in the alphabet
1
count table
[1, 5, 2, 2, 2]
position of letter in first column
11
new letter is A
letter is 1th place in the alphabet
4
count t

### String search using the BWT

Using a similar technique as in the efficient inverse BWT, it is possible to detect the presence of a substring $T$ in a string $S$ using the following principle

__initialization__

+ $L \leftarrow BWT(S)$
+ $F \leftarrow$ First column of the BW table
+ $e \leftarrow 1$
+ $f \leftarrow |F|$
+ $i \leftarrow |T|$

__while $e < f$ and $i>0$:__
 
+ $X\leftarrow T[i]$
+ $r \leftarrow $ first position of $T[i]$ in L[e,f]
+ $s \leftarrow $ last position of $T[i]$ in L[e,f]
+ $e \leftarrow $ index of the r-th occurrence of $T[i]$ in F
+ $f \leftarrow $ index of the s-th occurrence of $T[i]$ in F
+ $i \leftarrow i-1$

__Question:__
+ It is possible to avoid computing and storing the first column, propose a method to do this (take a look to the Efficient Inverse Burrows Wheeler Transform)
+ Program this function in python
+ Assuming that we store the index of the characters in the original string (as in a suffix array), it is possible to use the previous method to detect the location of the instances of the substring $T$ in $S$.

In [110]:
def pattern_matching_BWT(S,pattern):
    """
    Search a patter in a String using the BWT
    
    Args:
        S (str): string
        pattern (str): pattern
    
    Return:
        bool: true if the pattern is in the string    
    """
    pattern_in_S = False
    bwt=BWT(S)
    print("bwt")
    print(bwt)
    localisation, count= localisation_count(bwt)
    
    i=len(pattern)-1 #position de la dernière lettre du pattern
    alphabet={'$':0, 'A':1,'C':2,'G':3, 'T':4}
    la=[]
    lb=[]
    
    
    letter=pattern[i]
    # initialisation des "lettres intéressantes"
    for j in range(len(bwt)):
        if bwt[j]==letter:
            la.append(j)
            
    while i!=0:
        letter=pattern[i]
        print("on est à la lettre "+str(i)+ " du pattern, qui est "+ str(letter))
        i-=1
        k=0
        print("les positions des lettres interessantes sont: ")
        print(la)
        # on cherche les élèments qui correspondent à la lettre suivante de notre pattern
        for k in range(len(la)):
            index=la[k]
            right_letter=letter
            letter, position=left_corresponding_letter(right_letter, alphabet, bwt, count, localisation, index)
            
            if letter==pattern[i]:
                lb.append(position)
                print("la lettre suivante (qui est "+ str(letter)+") est à la position "+ str(position))
    
            if not lb:
                print("aucune lettre du texte ne correspond à la lettre suivante")
                return pattern_in_S
        la=lb
        lb=[]
                
        #on vérifie que la liste n'est pas vide
        
    
    pattern_in_S=True
    
    return pattern_in_S

In [111]:
if pattern_matching_BWT('ACATACAGATG','CATAC'):
    print("well done!")
else:
    print("wrong")
    print(pattern_matching_BWT('ACATACAGATG','CATAC'))

bwt
GT$CCGAATAAA
on est à la lettre 4 du pattern, qui est C
les positions des lettres interessantes sont: 
[3, 4]
la lettre suivante (qui est A) est à la position 6
on est à la lettre 3 du pattern, qui est A
les positions des lettres interessantes sont: 
[6]
la lettre suivante (qui est T) est à la position 1
on est à la lettre 2 du pattern, qui est T
les positions des lettres interessantes sont: 
[1]
la lettre suivante (qui est A) est à la position 10
on est à la lettre 1 du pattern, qui est A
les positions des lettres interessantes sont: 
[10]
la lettre suivante (qui est C) est à la position 4
well done!


### More material
https://www.youtube.com/watch?v=6BJbEWyO_N0&ab_channel=BenLangmead

https://www.youtube.com/watch?v=GWFb_C4IR14&ab_channel=BenLangmead

https://www.molgen.mpg.de/3708260/bwt_fm.pdf

https://www.coursera.org/lecture/algorithms-on-strings/using-bwt-for-pattern-matching-n8hIN